<a href="https://colab.research.google.com/github/YorkJong/vistock/blob/feature%2Fibd/notebooks/ibd_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install vistock from GitHub (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

In [3]:
%pip install "git+https://github.com/YorkJong/vistock.git@feature/ibd"

  Cloning https://github.com/YorkJong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-0kv7rg2t
  Running command git clone --filter=blob:none --quiet https://github.com/YorkJong/vistock.git /tmp/pip-req-build-0kv7rg2t
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/YorkJong/vistock.git to commit 915297b6c1f0b983623aa8661b83ec3e0d0fcec4
  Preparing metadata (setup.py) ... done


#### Common Functions

In [11]:
import os
from datetime import datetime

from vistock import ibd
from vistock import tw
from vistock.stock_indices import get_tickers


def rank(code, min_percentile=90, period='2y',
         tickers_getter=get_tickers, ref_ticker='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)
    rank_stock, rank_indust = ibd.rankings(tickers, period=period,
                                           ref_ticker=ref_ticker)

    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    print('\nStock Rankings:')
    #display(rank_stock)
    top_stocks = rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile]
    display(top_stocks)
    num_rows, _ = top_stocks.shape
    print(f'\nnumber of filtered tickers: {num_rows}')
    top_stock_list = list(top_stocks["Ticker"])
    print(top_stock_list)

    print('\n\nIndustry Rankings:')
    top_industries = rank_indust[rank_indust[ibd.TITLE_PERCENTILE] >= min_percentile]
    display(top_industries)
    num_rows, _ = top_industries.shape
    print(f'\nnumber of filtered sectors: {num_rows}')

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Save to CSV
    print("\n\n***")
    today = datetime.now().strftime('%Y%m%d')
    for table, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_{today}.csv'
        table.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")


### Usage Explanation

##### Parameters
source (選擇要分析的股票來源):
- The source of stocks to analyze

min_percentile (最小百分位)
- The minimum percentile for a stock to be included in the rankings.

period (歷史資料時間範圍)：
- The period for which to fetch historical data.

### IBD RS Rating and Ranking

In [6]:
source = "All Indices" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "PHLX Semiconductor", "All Indices"]
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'all'
}

rank(code_from_name[source], min_percentile, period)

ERROR:yfinance:BRK.B: No data found, symbol may be delisted
ERROR:yfinance:ARM: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:VLTO: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:$BF.B: possibly delisted; No price data found  (period=2y)


$BF.B: possibly delisted; No price data found  (period=2y)


ERROR:yfinance:SW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']
ERROR:yfinance:GEV: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:SOLV: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']



Stock Rankings:


,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
441,NVDA,Technology,Semiconductors,153.11,189.40,171.60,176.14,99,99,99,99,1
258,VST,Utilities,Utilities - Independent Power Producers,152.26,192.90,238.77,138.33,99,99,99,97,2
134,IRM,Real Estate,REIT - Specialty,147.73,123.27,113.08,105.08,99,95,85,71,3
429,FICO,Technology,Software - Application,143.04,130.98,118.38,131.71,99,97,91,96,4
204,GDDY,Technology,Software - Infrastructure,141.21,125.73,134.82,119.81,99,96,98,93,5
173,HWM,Industrials,Aerospace & Defense,135.80,122.79,137.10,114.55,99,94,98,88,6
3,MMM,Industrials,Conglomerates,135.23,104.05,111.20,83.24,98,79,82,16,7
181,TRGP,Energy,Oil & Gas Midstream,133.83,124.80,121.58,101.89,98,96,93,64,8
458,AXON,Industrials,Aerospace & Defense,132.28,99.95,114.31,113.13,98,72,88,86,9
92,GE,Industrials,Aerospace & Defense,130.72,124.68,146.89,121.87,98,95,98,94,10



number of filtered tickers: 52
['NVDA', 'VST', 'IRM', 'FICO', 'GDDY', 'HWM', 'MMM', 'TRGP', 'AXON', 'GE', 'UHS', 'KKR', 'TSM', 'NRG', 'ANET', 'MHK', 'NTAP', 'ISRG', 'AVGO', 'CEG', 'TYL', 'MPWR', 'EFX', 'K', 'CBRE', 'WELL', 'VTR', 'RCL', 'BRO', 'MSI', 'CTAS', 'MCK', 'EXR', 'IP', 'META', 'CFG', 'GS', 'TMUS', 'GEN', 'PGR', 'COST', 'EBAY', 'HCA', 'RTX', 'HIG', 'TT', 'FITB', 'PM', 'DHI', 'AMT', 'ICE', 'FSLR']


Industry Rankings:


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
84,Utilities - Independent Power Producers,Utilities,140.50,164.12,207.76,127.36,"VST,NRG",99,99,99,98,1
26,REIT - Healthcare Facilities,Real Estate,118.69,104.13,100.62,92.65,"WELL,VTR",98,92,64,33,2
12,Medical Care Facilities,Healthcare,118.25,100.85,109.08,106.12,"UHS,HCA,DVA",97,90,92,82,3
60,Oil & Gas Midstream,Energy,118.14,107.73,110.19,95.14,"TRGP,WMB,KMI,OKE",96,95,94,47,4
3,Conglomerates,Industrials,115.86,98.83,102.34,86.30,"MMM,HON",95,82,70,15,5
1,Tobacco,Consumer Defensive,114.31,98.55,97.81,84.08,"PM,MO",94,78,55,11,6
10,Residential Construction,Consumer Cyclical,111.94,100.00,107.38,112.54,"DHI,PHM,NVR,LEN",93,87,83,95,7
49,Consulting Services,Industrials,110.02,97.88,95.88,104.74,"EFX,VRSK",92,74,44,75,8
30,Computer Hardware,Technology,109.90,134.40,140.70,173.51,"ANET,NTAP,STX,HPQ,SMCI,WDC",91,98,98,99,9
28,Aerospace & Defense,Industrials,108.48,97.81,107.56,101.85,"HWM,AXON,GE,RTX,LMT,TDG,LHX,GD,NOC,HII,TXT,BA",90,73,87,65,10



number of filtered sectors: 10


***
Your "all_stocks_2y_20240807.csv" is in the "out" folder.
Your "all_industries_2y_20240807.csv" is in the "out" folder.
***



### IBD RS Rating and Ranking for Taiwan Stocks

In [10]:
from vistock import tw

source = "上市" #@param ["上市", "上櫃", "興櫃", "全部"]
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
period = "1y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '興櫃': 'ESB',
    '全部': 'all'
}

rank(code_from_name[source], min_percentile, period,
     tickers_getter=tw.get_tickers, ref_ticker='^TWII')

ERROR:yfinance:00939.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:00940.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:00941.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:00943.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:00944.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:00945B.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:00946.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:00947.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:00949.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:00951.TW: P


Stock Rankings:


,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1012,6442.TW,Technology,Electronic Components,377.79,295.97,152.21,106.91,99.0,99,99,91,1
470,2365.TW,Technology,Computer Hardware,323.97,237.26,113.72,112.27,99.0,99,92,95,2
1132,8374.TW,Industrials,Industrial Distribution,302.20,279.67,95.14,96.24,99.0,99,56,26,3
860,4562.TW,Industrials,Specialty Industrial Machinery,300.94,297.46,138.79,96.20,99.0,99,98,25,4
455,2348.TW,Real Estate,Real Estate Services,274.30,176.60,154.19,101.55,99.0,98,99,78,5
...,...,...,...,...,...,...,...,...,...,...,...,...
540,2465.TW,Technology,Computer Hardware,114.57,129.70,140.36,114.24,90.0,95,98,96,114
574,2516.TW,Industrials,Engineering & Construction,114.48,91.83,94.25,106.97,90.0,63,52,91,115
206,1316.TW,Real Estate,Real Estate Services,114.48,101.88,118.65,108.78,90.0,80,95,93,116
600,2609.TW,Industrials,Marine Shipping,114.24,110.19,106.15,101.85,90.0,88,85,79,117



number of filtered tickers: 118
['6442.TW', '2365.TW', '8374.TW', '4562.TW', '2348.TW', '2543.TW', '3535.TW', '2359.TW', '5484.TW', '9906.TW', '2438.TW', '3312.TW', '2524.TW', '1225.TW', '2467.TW', '1540.TW', '3047.TW', '6446.TW', '6117.TW', '2537.TW', '1438.TW', '8467.TW', '8028.TW', '3450.TW', '1213.TW', '2349.TW', '2429.TW', '1519.TW', '6215.TW', '3686.TW', '2540.TW', '1436.TW', '6706.TW', '2615.TW', '3030.TW', '6807.TW', '2464.TW', '3048.TW', '2374.TW', '9946.TW', '2486.TW', '9105.TW', '6869.TW', '3013.TW', '3376.TW', '5225.TW', '2236.TW', '2243.TW', '3229.TW', '1597.TW', '3653.TW', '8114.TW', '6605.TW', '6449.TW', '1514.TW', '2597.TW', '6139.TW', '2613.TW', '1235.TW', '6197.TW', '4566.TW', '1522.TW', '6873.TW', '2548.TW', '5519.TW', '1560.TW', '4545.TW', '3665.TW', '6177.TW', '1463.TW', '4967.TW', '3018.TW', '2367.TW', '9902.TW', '1442.TW', '2504.TW', '2611.TW', '6152.TW', '2476.TW', '2317.TW', '2392.TW', '1725.TW', '2330.TW', '3583.TW', '1503.TW', '9955.TW', '3593.TW', '2312.TW'

,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
46,Industrial Distribution,Industrials,161.23,158.30,96.24,97.04,"8374.TW,911608.TW,2373.TW",99.0,98,59,27,1
72,Utilities—Renewable,Utilities,122.93,130.89,106.74,107.59,"6869.TW,6873.TW,6806.TW",98.0,97,93,97,2
19,Real Estate—Diversified,Real Estate,118.96,114.96,114.27,104.21,"1438.TW,9946.TW,2545.TW,2520.TW,2547.TW",96.0,94,99,96,3
48,Engineering & Construction,Industrials,118.77,107.10,105.41,101.93,"2543.TW,9906.TW,2597.TW,6139.TW,5519.TW,3018.T...",94.0,88,89,89,4
18,Real Estate—Development,Real Estate,114.45,106.22,110.89,100.04,"2524.TW,2537.TW,1436.TW,2548.TW,6177.TW,2539.T...",92.0,86,98,70,5
10,Real Estate Services,Real Estate,112.99,103.14,109.06,101.98,"2348.TW,2540.TW,9902.TW,1442.TW,1316.TW,2530.T...",91.0,85,97,91,6



number of filtered sectors: 6


***
Your "TWSE_stocks_1y_20240807.csv" is in the "out" folder.
Your "TWSE_industries_1y_20240807.csv" is in the "out" folder.
***

